In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

# Configurar o dispositivo (Usar GPU se disponível)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando: {device}")

# 1. Transformações: Redimensionamos para 224x224 para a ResNet50
transform = transforms.Compose([
    transforms.Resize(224), 
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

# 2. Baixar e carregar o CIFAR-10
train_set = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_set = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

classes = ('avião', 'carro', 'pássaro', 'gato', 'veado', 'cachorro', 'sapo', 'cavalo', 'navio', 'caminhão')

Usando: cuda


100%|██████████| 170M/170M [00:01<00:00, 85.8MB/s]


In [2]:
# Carregar ResNet50 pré-treinada
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

# Congelar os pesos 
for param in model.parameters():
    param.requires_grad = False

# Substituir a última camada (FC) para 10 classes
model.fc = nn.Linear(model.fc.in_features, 10)
model = model.to(device)

# Definir erro e otimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 192MB/s]


In [3]:
epochs = 3 # Apenas 2 para exemplo rápido

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Época {epoch+1} - Loss: {running_loss/len(train_loader):.4f}")

Época 1 - Loss: 0.7987
Época 2 - Loss: 0.5962
Época 3 - Loss: 0.5457


In [4]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    image, label = test_loader.dataset[0]
    images = image.to(device)
    images = images.unsqueeze(0) 
    output = model(images)
    _, predicted = torch.max(output, 1)

print("Classe real:     ", classes[label])
print("Classe predita:  ", classes[predicted.item()])

print(f"Acurácia no Teste: {100 * correct / total:.2f}%")

Classe real:      gato
Classe predita:   gato
Acurácia no Teste: 81.56%
